# Hash Maps

A hash map is a data structure that provides a lookup in $O(1)$. It stores key-value pairs in which the keys are hashed to integers using a hash function. Ideally, the hash function should map each possible key to a unique index in the array. In practice, this is not possible. Collisions occur when the hash function maps two different keys to the same index. There are two common ways to deal with collisions: chaining and open addressing. This notebook will show implementations of both.

## Hash Functions

A hash function is a function that maps a key to an integer. The hash function should be fast to compute and should distribute the keys uniformly across the array. The hash function should also be deterministic, meaning that the same key should always map to the same integer. There are several strategies for hashing numbers, strings, and objects. The example below shows a simple hash function which sums the ASCII values of the characters in a string.

In [1]:
def hash(key, size):
    # Convert key to ascii values
    key = str(key).encode()
    return sum(key) % size

buckets = [None] * 10
buckets[hash('hello', 10)] = 'world'

print(hash('hello', 10))
print(buckets[hash('hello', 10)])

2
world


The hash function above is often referred to as **lose lose**. It is evaluated in this [excellent StackOverflow post](https://softwareengineering.stackexchange.com/questions/49550/which-hashing-algorithm-is-best-for-uniqueness-and-speed?newreg=72a6cd970f244b619fe0564ccea3dee7) along with many other hash functions. A more robust hash function for strings is the [djb2](http://www.cse.yorku.ca/~oz/hash.html) hash function. This hash function is used in the Python dictionary implementation.


In [2]:
def djb2(key, size):
    hash_value = 5381
    key = str(key).encode()
    for char in key:
        hash_value = ((hash_value << 5) + hash_value) + char
    return hash_value % size

buckets = [None] * 10
buckets[djb2('hello', 10)] = 'world'

print(djb2('hello', 10))
print(buckets[djb2('hello', 10)])

1
world


Now that we have a hash function, let's create a ~HashMap~ class that integrates the hash function. We will test it out on a more robust example which will reveal the need for collision resolution.

In [4]:
class HashMap:
    def __init__(self, size):
        self.size = size
        self.buckets = [None] * self.size

    # def hash(self, key):
    #     hash_value = 5381
    #     key = str(key).encode()
    #     for char in key:
    #         hash_value = ((hash_value << 5) + hash_value) + char
    #     return hash_value % self.size

    def hash(key, size):
        # Convert key to ascii values
        key = str(key).encode()
        return sum(key) % size
    
    def insert(self, key, value):
        index = self.hash(key)

        if self.buckets[index] is None:
            self.buckets[index] = value
        else:
            raise Exception(f'Collision at index {index}')

    def find(self, key):
        index = self.hash(key)
        return self.buckets[index]

    def __setitem__(self, key, value):
        self.insert(key, value)

    def __getitem__(self, key):
        return self.find(key)
    
    def __repr__(self):
        return str(self.buckets)
    
    def __str__(self):
        return str(self.buckets)
    
hash_map = HashMap(10)
hash_map.insert('Naomi', 100)
hash_map.insert('Amos', 200)
hash_map.insert('Alex', 300)
hash_map.insert('James', 400)
hash_map.insert('Bobbie', 500)

Exception: Collision at index 1

# Handing Collisions

Uh oh! We had a collision when inserting `Alex` in the example above. How can we resolve this? There are two general methods of dealing with collisions: chaining and open addressing. Chaining involves using a secondary data structure to store the collisions and open addressing involves finding an open slot in the array to store the collision.

## Chaining

First, we will take a look at chaining. For this, let's use a naive approach. If a collision occurs, we will store the `(key, value)` pair in a list at the index of the array. A more efficient approach would be to use a red-black tree. This would allow for $O(\log n)$ lookup instead of $O(n)$ lookup.

In [7]:
class HashMap:
    def __init__(self, size):
        self.size = size
        self.buckets = [None] * self.size

    # def hash(self, key):
    #     hash_value = 5381
    #     key = str(key).encode()
    #     for char in key:
    #         hash_value = ((hash_value << 5) + hash_value) + char
    #     return hash_value % self.size

    def hash(self, key):
        # Convert key to ascii values
        key = str(key).encode()
        return sum(key) % self.size
    
    def insert(self, key, value):
        index = self.hash(key)

        if self.buckets[index] is None:
            self.buckets[index] = [(key, value)]
        else:
            self.buckets[index].append((key, value))

    def find(self, key):
        index = self.hash(key)

        if self.buckets[index] is not None:
            for item in self.buckets[index]:
                if item[0] == key:
                    return item[1]

        return None
    
    def __setitem__(self, key, value):
        self.insert(key, value)

    def __getitem__(self, key):
        return self.find(key)
    
    def __repr__(self):
        return str(self.buckets)
    
    def __str__(self):
        return str(self.buckets)
    
hash_map = HashMap(10)
hash_map.insert('Naomi', 100)
hash_map.insert('Amos', 200)
hash_map.insert('Alex', 300)
hash_map.insert('James', 400)
hash_map.insert('Bobbie', 500)

print(hash_map['Bobbie'])
print(hash_map['Alex'])

print(hash_map)

500
300
[[('Naomi', 100), ('Amos', 200)], None, None, None, [('Alex', 300)], None, [('James', 400)], None, None, [('Bobbie', 500)]]
